In [7]:
###################################### ESTRATEGIA BB
###################################################################################################################
import modulos as md
import numpy as np
import constantes as cons

lista=[
'KNCUSDT',
'ARKMUSDT',
'BELUSDT',
'AGLDUSDT',
'STXUSDT',
'FTMUSDT',
'XRPUSDT',
'SRMUSDT',
'SUSHIUSDT',
'SXPUSDT',
'ALICEUSDT',
'PENDLEUSDT',
'MATICUSDT',
'EOSUSDT',
'CRVUSDT',
'OMGUSDT',
'EDUUSDT'
]
lista_filtrada = []

lista=md.lista_de_monedas ()
#lista=['THETAUSDT']

for symbol in lista:    
    try:
        #data = md.estrategia_santa(symbol,tp_flag = True)
        data = md.estrategia_triangulos(symbol, tp_flag = True, print_lines_flag = False)
        #######################################################################
        resultado = md.backtesting(data, plot_flag = False)
        if ((resultado['Profit Factor'] > 2 or np.isnan(resultado['Profit Factor'])) and (resultado['Return [%]']/resultado['# Trades']) >=0.33):
            lista_filtrada.append(symbol)
            print(f"{symbol} - Return [%]: {md.truncate(resultado['Return [%]'],2)}% - # Trades: {resultado['# Trades']} - Profit Factor: {resultado['Profit Factor']} - Win Rate [%]: {resultado['Win Rate [%]']}")
    except Exception as falla:
        pass
for symbol in lista_filtrada:
    md.printandlog(cons.nombrelog,symbol,pal=1)
#md.dibuja_patrones_triangulos (data,998)    


In [4]:
md.dibuja_patrones_triangulos (data,992) 

0.9454404901450448 -1.0


linea superior. X: [ 984.  988. 1003.] - y: [0.77   0.768  0.7605]
linea inferior. X: [ 972.  973.  977.  985.  990. 1005.] - y: [0.75471452 0.75529364 0.75761011 0.76224307 0.76513866 0.77382545]
